# Project 5 - 03 Predictive model


Utilizing the [Coronavirus Disease 2019 (COVID-19) Clinical Data Repository](https://covidclinicaldata.org/), we will examine the influence of individual symptoms on whether a COVID test will be positive. We hope that the understanding gained will help frontline medical works with limited time and resources to triage cases and determine initial steps in creating treatment plans.


We have 3 notebooks as blow:
* 01, 02 - Designed to investigate association between COVID and the patients' symptoms
* 03 - Designed to create a classification model which predicts whether someone has COVID

This is 3rd notebook, and we want to find as accurate model as possible

The dataset has `covid19_test_results`, which says positive or negative based on patients' test results so we set this column as target value.

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.linear_model import LogisticRegression
np.random.seed(42)

---

## Assemble DataFrame and cleaning

We'll want all the individual `.csv` files in one data frame. Most of the files contain the Carbon Health testing data for one week with the exception of the first file which contains data for one month. Fortunately, the compiler of this data has maintained consistency in the features and data logging for the project, so we should be able to jump right into cleaning.   

In [2]:
# Get entire data
df = pd.concat([pd.read_csv(f'../data/original_data/{file}') for file in os.listdir('../data/original_data/')], ignore_index=True)

In [3]:
print(df.shape)
df.isnull().sum()

(93995, 46)


batch_date                           0
test_name                            0
swab_type                            0
covid19_test_results                 0
age                                  0
high_risk_exposure_occupation      169
high_risk_interactions           24827
diabetes                             0
chd                                  0
htn                                  0
cancer                               0
asthma                               0
copd                                 0
autoimmune_dis                       0
smoker                               0
temperature                      46453
pulse                            45716
sys                              47472
dia                              47472
rr                               52547
sats                             46460
rapid_flu_results                93741
rapid_strep_results              93604
ctab                             58528
labored_respiration              45248
rhonchi                  

**Entire DataFrame shapes 94684 rows and 47 columns. There are some columns which have over 90000 columns. This is too many to keep**

In [4]:
# Drop columns which has more than 90000 missing values
# 'rapid_flu_results', 'rapid_strep_results', 'sob_severity', 'cxr_findings', 'cxr_impression', 'cxr_label', 'cxr_link']
df.drop(df.columns[df.isnull().sum()>90000], axis=1, inplace=True)

---

## EDA
There are several columns which can be converted to numeric, and we will binarize all of booleans.
Some object columns, `batch_date`, `test_name`, `swab_type`, are left but we believe these features will not help prediction accuracy so that we will drop it. 

In [5]:
# Binarize booleans through entire DataFrame
df = df.replace(True, 1).replace(False, 0)

In [6]:
# Convert cough_severity to numeric
df = df.replace('Mild', 1).replace('Moderate', 2).replace('Severe', 3)

In [7]:
# Convert test result to numeric
df['covid19_test_results'] = df['covid19_test_results'].map({'Positive': 1, 'Negative': 0})

In [8]:
# Dumify and make Dataframe which contains only object columns.
# These columns look not important but I keep it just in case
dummy = pd.get_dummies(data=df, columns=['test_name', 'swab_type'])
dummy['batch_date'] = pd.to_datetime(df['batch_date'])
df.drop(columns={'batch_date', 'test_name', 'swab_type'}, inplace=True)

### Find baseline

In [9]:
df['covid19_test_results'].value_counts(1)

0    0.986031
1    0.013969
Name: covid19_test_results, dtype: float64

**We found this dataset is very imbalanced as there are many negative results compared to positive results so we will conduct oversample from positive rows and reduce negative rows**

We will make DataFrame for each of positive result and negative result, then process EDA on each DataFrame separately

In [10]:
# Dataframe for positives and negatives
positive = df[df['covid19_test_results']==1]
negative = df[df['covid19_test_results']==0]

In [11]:
negative.shape

(92682, 36)

In [12]:
positive.shape

(1313, 36)

In [13]:
negative.isnull().sum()

covid19_test_results                 0
age                                  0
high_risk_exposure_occupation      164
high_risk_interactions           24592
diabetes                             0
chd                                  0
htn                                  0
cancer                               0
asthma                               0
copd                                 0
autoimmune_dis                       0
smoker                               0
temperature                      46211
pulse                            45485
sys                              47223
dia                              47223
rr                               52202
sats                             46223
ctab                             58100
labored_respiration              45012
rhonchi                          69929
wheezes                          65884
days_since_symptom_onset         77487
cough                               15
cough_severity                   87347
fever                    

### Clean `negative`
Negative DataFrame is much huger than positive DataFrame. In order to scale `negative` to same size as `positive`, We will drop rows which have missing values

In [14]:
# We will drop and decrease negative rows:
# Drop rows which has a missing value in any columns which have less than 80000 missing values
# The reason of 80000 is to avoid to decrease DataFrame size too much
for column in negative.columns[negative.isnull().sum()<80000]:
    negative.drop(negative[column][negative[column].isnull()].index, inplace=True)

C:\Users\Nao\.conda\envs\dsi\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
negative.shape

(5322, 36)

In [16]:
# Missing values of negative
negative.isnull().sum()[negative.isnull().sum()>0]

cough_severity    3453
er_referral       4365
dtype: int64

2 columns still have missing values.
1. We will drop `er_refferal` as it has too many missing values.
1. We drop rows which have missing values in `cough_severity`

In [17]:
# Drop er_referral as too many missing value
negative.drop(columns={'er_referral'}, inplace=True)

C:\Users\Nao\.conda\envs\dsi\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
# Drop rows which has missing value at cough_severity
negative.drop(negative['cough_severity'][negative['cough_severity'].isnull()].index, inplace=True)

In [19]:
negative.shape

(1869, 35)

In [20]:
positive.shape

(1313, 36)

### Clean `positive`
`positive` is small DataFrame so we do not want to drop many rows.

In [21]:
# Missing value for positive
positive.isnull().sum()[positive.isnull().sum()>0]

high_risk_exposure_occupation      5
high_risk_interactions           235
temperature                      242
pulse                            231
sys                              249
dia                              249
rr                               345
sats                             237
ctab                             428
labored_respiration              236
rhonchi                          722
wheezes                          623
days_since_symptom_onset         643
cough_severity                   937
fever                            207
sob                                5
diarrhea                           5
fatigue                            5
headache                           5
loss_of_smell                      5
loss_of_taste                      5
runny_nose                         5
muscle_sore                        5
sore_throat                        5
er_referral                      998
dtype: int64

1. **We will drop `er_refferal` as we did on `negative`**
1. **There are many columns which have only 5 missing values. This is small enough to drop**

In [22]:
# Drop er_referral as we dropped this column from negative
positive.drop(columns={'er_referral'}, inplace=True)

C:\Users\Nao\.conda\envs\dsi\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
# Drop rows which has missing value at below columns which has 5 missing values
# 'high_risk_exposure_occupation', 'sob', 'diarrhea', 'fatigue', 'headache', 'loss_of_smell', 'loss_of_taste', 'runny_nose', 'muscle_sore', 'sore_throat'
positive.drop(positive['sob'][positive['sob'].isnull()].index, inplace=True)

In [24]:
# Missing value for positive
positive.isnull().sum()[positive.isnull().sum()>0]

high_risk_interactions      230
temperature                 238
pulse                       227
sys                         245
dia                         245
rr                          341
sats                        233
ctab                        423
labored_respiration         235
rhonchi                     717
wheezes                     618
days_since_symptom_onset    638
cough_severity              932
fever                       202
dtype: int64

In [25]:
positive.shape

(1308, 35)

In [26]:
# Reset index
positive.reset_index(drop=True, inplace=True)
negative.reset_index(drop=True, inplace=True)

### EDA on `positive`
We will fill out missing values for `positive` with 2 methods.
* For columns which have only 1 or 0, we will randomly generate binomial values with probability based on each column
    * If portion of 1 in a column of `positive` is 20 %, generate binomial values with 20% probability and fill the column with the values
* For other columns, we will randomly generate normally distributed values with mean and std based each column
    * If a columns of `positive` has mean of 10 and standard deviation of 3, generate noramlly distributed values with the mean and std
    * This would not be very good for all of columns but we guess this is better than just fill those with its mean

In [27]:
# Binary column and continuous column
bi_col = ['high_risk_interactions', 'ctab', 'labored_respiration', 'wheezes', 'fever', 'rhonchi']
con_col = ['temperature', 'pulse', 'sys', 'dia', 'rr', 'sats', 'days_since_symptom_onset', 'cough_severity']

In [28]:
# Function to fill missing values of binary columns for positive
# Fill 1 at percentage of 1 which are given by other values in same column
def fill_binary(column):
    missing_index = positive[column][positive[column].isnull()].index # Index which has missing value at 'column'
    prob = positive[column].value_counts(1).iloc[1] # Get portion of normalized 1
    fil = np.random.binomial(1, prob, len(missing_index)) # Binomial list at probability of prob
    for index, value in zip(missing_index, fil):
        positive.loc[index, column] = value # Fill missing values with above binomial list

In [29]:
# Same above but for continuous columns
# Filling values are given by normalized distribution with mean and std of each column
def fill_con(column):
    missing_index = positive[column][positive[column].isnull()].index
    prob = positive[column].value_counts(1).iloc[1]
    fil = np.random.normal(loc=positive.describe()[column].loc['mean'], scale=positive.describe()[column].loc['std'], size=len(missing_index))
    for index, value in zip(missing_index, fil):
        positive.loc[index, column] = value

In [30]:
# Fill missing values for binary columns
for col in bi_col:
    fill_binary(col)

C:\Users\Nao\.conda\envs\dsi\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [31]:
# Fill missing values for continuouse columns
for col in con_col:
    fill_con(col)

### Resample positive
`positive` is still small compared to `negative` so that we will resample `positive`

In [32]:
positive_lst = []
positive_lst.append(positive.sample(positive.shape[0], replace=True))
positive_lst.append(positive)

In [33]:
# Resampled positive DataFrame
boost_positive = pd.concat(positive_lst)
boost_positive.shape

(2616, 35)

---

## Modeling
We will create following models
* LogisticRegrssion
* Neural network
* SVM
* VotingClassifier

In [34]:
# Concatinate positive and negative
temp = pd.concat([boost_positive, negative])

In [35]:
X = temp.drop('covid19_test_results', axis=1)
y = temp['covid19_test_results']

In [36]:
X.shape

(4485, 34)

In [37]:
# Baseline
y.value_counts(1)

1    0.583278
0    0.416722
Name: covid19_test_results, dtype: float64

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [39]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

### SVM

In [40]:
svc = SVC()

p_grid = {
    'C':np.linspace(0, 5, 20),
    'kernel':['linear', 'rbf', 'polynomial','sigmoid'],
    'degree':[1, 2, 3, 4]
}

gssvc = GridSearchCV(estimator=svc, param_grid=p_grid, cv=5, n_jobs=6)
gssvc.fit(Z_train, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=6,
             param_grid={'C': array([0.        , 0.26315789, 0.52631579, 0.78947368, 1.05263158,
       1.31578947, 1.57894737, 1.84210526, 2.10526316, 2.36842105,
       2.63157895, 2.89473684, 3.15789474, 3.42105263, 3.68421053,
       3.94736842, 4.21052632, 4.47368421, 4.73684211, 5.        ]),
                         'degree': [1, 2, 3, 4],
                         'kernel': ['linear', 'rbf', 'polynomial', 'sigmoid']})

In [41]:
best_svm = gssvc.best_estimator_
best_svm.fit(Z_train, y_train)

SVC(C=3.1578947368421053, degree=1)

In [42]:
print('train', best_svm.score(Z_train, y_train))
print('test', best_svm.score(Z_test, y_test))

train 0.9271483794231341
test 0.8832442067736186


### Neural network

In [43]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(Z_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
results = model.fit(Z_train, y_train, epochs=30, verbose=0, batch_size=256, validation_data=(Z_test, y_test)) 

In [44]:
print(model.evaluate(Z_test, y_test))
print('train', results.history['acc'][-1])
print('test', results.history['val_acc'][-1])

36/36 [==============================] - 0s 471us/step - loss: 0.0990 - acc: 0.8672
[0.09897058457136154, 0.8672014474868774]
train 0.9030627608299255
test 0.8672014474868774


### LogisticRegression

In [45]:
lr = LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)
print('train', lr.score(X_train, y_train))
print('test', lr.score(X_test, y_test))

train 0.8218852215283973
test 0.8315508021390374


### VotingClassifier

In [46]:
# This cell needs long time to run
knn_pipe = Pipeline([
    ('ss' , StandardScaler()),
    ('knn', KNeighborsClassifier())
])
vote = VotingClassifier([
    ('ada', AdaBoostClassifier()),
    ('gb', GradientBoostingClassifier()),
    ('dt', DecisionTreeClassifier()),
    ('knn_pipe', knn_pipe),
    ('xgb', XGBClassifier())
])
params = {
    'ada__base_estimator' : [DecisionTreeClassifier(max_depth=2), None],
    'ada__n_estimators' : [95, 100, 125],
    'gb__n_estimators' : [20, 30, 50],
    'dt__max_depth' : [4, 5, 6],
    'knn_pipe__knn__n_neighbors' : [3, 4, 5],
    'xgb__gamma': [0, 1, 2]
}
gs = GridSearchCV(vote, param_grid=params, cv=5, n_jobs=6)
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=VotingClassifier(estimators=[('ada',
                                                     AdaBoostClassifier()),
                                                    ('gb',
                                                     GradientBoostingClassifier()),
                                                    ('dt',
                                                     DecisionTreeClassifier()),
                                                    ('knn_pipe',
                                                     Pipeline(steps=[('ss',
                                                                      StandardScaler()),
                                                                     ('knn',
                                                                      KNeighborsClassifier())])),
                                                    ('xgb',
                                                     XGBClassifier(base_score=None,
              

In [47]:
print(gs.best_score_)
best_estimator = gs.best_estimator_
best_estimator.fit(X_train, y_train)

0.9119865739758012


VotingClassifier(estimators=[('ada',
                              AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                                                 n_estimators=125)),
                             ('gb',
                              GradientBoostingClassifier(n_estimators=50)),
                             ('dt', DecisionTreeClassifier(max_depth=6)),
                             ('knn_pipe',
                              Pipeline(steps=[('ss', StandardScaler()),
                                              ('knn',
                                               KNeighborsClassifier(n_neighbors=3))])),
                             ('xgb',
                              XGBClassifier(base_score=...
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                    

In [48]:
# Score
print('train', best_estimator.score(X_train, y_train))
print('test', best_estimator.score(X_test, y_test))

train 0.975319655069878
test 0.93048128342246


In [49]:
# Make DataFrame to store scores of each estimator
scores = {}
for name, estimator in best_estimator.named_estimators_.items():
    scores[name+'_train'] = estimator.score(X_train, y_train)
    scores[name+'_test'] = estimator.score(X_test, y_test)

In [50]:
scores['voting_train'] = best_estimator.score(X_train, y_train)
scores['voting_test'] = best_estimator.score(X_test, y_test)
models = {}

models['1'] = scores
pd.DataFrame.from_dict(models, orient='index')

,ada_train,ada_test,gb_train,gb_test,dt_train,dt_test,knn_pipe_train,knn_pipe_test,xgb_train,xgb_test,voting_train,voting_test
1,0.977401,0.926025,0.883437,0.871658,0.868272,0.849376,0.90455,0.826203,1.0,0.934938,0.97532,0.930481


---

## Conclustion

Our best model is VotingClassifer

> * Train accuracy **97%**
> * Test accuracy **93%**

|VotingClassifier parameter|
|-|
|**AdaBoostClassifier**(base_estimator=**DecisionTreeClassifier(max_depth=2)**, n_estimators=125)|
|**GradientBoostingClassifier**(n_estimators=50)|
|**DecisionTreeClassifier**(max_depth=6)|
|**Pipeline**(steps=[('ss', StandardScaler()), ('knn', **KNeighborsClassifier(n_neighbors=3)**)])|
|**XGBClassifier**|

## Recommendation and next step

This model predicts at 93% accuracy so that we can reccomend frontline medical workers to use this model for brief screening of the potential patients. They would prioritize the patiets who got positive from this model to take diagnosis more carefully. Also they can group the patients based on the result whether positive or negative so that they could reduce infection onsite.

Although our predictive models are very accurate, they probably aren’t accurate enough for  the medical field or a possible life and death determination.


In order to obtain better model, we could try other combination of classifiers. VotingClassifier needs much time to calculate so that we did not try out many various of hyperparemeters but if we had more time or better machine, we would figure out better heperparameters and classifiers